# 신경망

신경망은 torch.nn 패키지를 사용하여 생성 가능 <br>
nn.Module은 계층(layer)과 output을 반환하는 forward(input) 메서드를 포함

[신경망의 일반적인 학습 과정] <br>
- 학습 가능한 매개변수(weight)를 갖는 신경망 정의
- 데이터셋 입력을 반복
- 입력을 신경망에서 전파
- 손실을 계산
- 변화도(gradient)를 신경망의 매개변수들에 역으로 전파
- 신경망의 가중치를 갱신. 새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)

## 신경망 정의하기

In [6]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 6개, 5x5의 정사각 컨볼루션 행렬
        # 컨볼루션 커널 정의
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Affine 연산: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5x*5는 이미지 차원에 해당
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # (2,2) 크기 윈도우에 대해 맥스 풀링 (max pooling)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 크기가 제곱수라면, 하나의 숫자만을 특정
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward 함수만 정의하고 나면, backward 함수는 autograd를 사용하여 자동으로 정의. forward 함수에서는 어떠한 tensor 연산을 사용해도 괜찮음

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1의 weight

10
torch.Size([6, 1, 5, 5])


임의의 32x32 입력값을 넣어보자.

In [7]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0630,  0.0362,  0.0139, -0.0397, -0.0189,  0.1129,  0.1000,  0.0278,
         -0.0744, -0.1079]], grad_fn=<AddmmBackward0>)


모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고 무작위 값으로 역전파를 함

In [8]:
net.zero_grad()
out.backward(torch.randn(1, 10))

요약: <br>
-torch.Tensor - backward() 같은 autograd 연산을 지원하는 다차원 배열 입니다. 또한 tensor에 대한 변화도를 갖고 있습니다.

-nn.Module - 신경망 모듈. 매개변수를 캡슐화(encapsulation)하는 간편한 방법 으로, GPU로 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공합니다.

-nn.Parameter - Tensor의 한 종류로, Module 에 속성으로 할당될 때 자동으로 매개변수로 등록 됩니다.

-autograd.Function - autograd 연산의 순방향과 역방향 정의 를 구현합니다. 모든 Tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor 를 생성하고 이력(history)을 인코딩 하는 함수들과 연결하고 있습니다.

## 손실함수

손실함수는 (output, target)을 한 쌍(pair)의 입력으로 받아, 출력(output)이 정답(target)으로부터 얼마나 멀리 떨어져 있는지 추정하는 값을 계산. <br>
nn 패키지에는 여러가지의 손실함수들이 존재

In [12]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1) # 출력과 같은 shape로 만듦
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0896, grad_fn=<MseLossBackward0>)


In [17]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## 역전파(Backprop)

오차(error)를 역전파하기 위해서는 loss.backward()만 해주면 된다. 기존에 계산된 변화도의 값을 누적시키고 싶지 않다면 기존에 계산된 변화도를 0으로 만드는 작업이 필요 → zero_grad

In [18]:
net.zero_grad() # 모든 매개변수의 변화도 버퍼를 0으로 만듦

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias grad after backward
tensor([ 0.0065,  0.0041, -0.0018, -0.0231, -0.0191,  0.0027])


## 가중치 갱신

확률적 경사 하강법

In [19]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [20]:
import torch.optim as optim

# Optimizer를 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)은 다음과 같습니다:
optimizer.zero_grad()   # 변화도 버퍼를 0으로
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 업데이트 진행